In [ ]:
#!pip3 install kfp --upgrade

In [ ]:
#!pip3 install tensorflow

In [ ]:
#!pip freeze

In [4]:
import kfp
from kfp.components import create_component_from_func

In [5]:
client = kfp.Client()

In [6]:
client

In [7]:
def add(a: float, b: float) -> float:
    return a + b

In [8]:
add_op = create_component_from_func(
    add, output_component_file="add_component.yaml"
)

In [9]:
#create and run pipeline

import kfp.dsl as dsl

@dsl.pipeline(
    name="Addition pipline",
    description="An example pipeline that perform addition calculator"
)

def add_pipeline(a='1', b='7'):
    first_add_task = add_op(a, 4)
    second_add_task = add_op(first_add_task.output, b)
    
arguments = {'a': '7', 'b': '8'}

client.create_run_from_pipeline_func(add_pipeline, arguments=arguments)

RunPipelineResult(run_id=720875bd-bedf-4dee-af81-21dbde6b4323)

In [10]:
from typing import NamedTuple

In [11]:
def multiple_return_values_example(a: float, b: float) -> NamedTuple(
    "ExampleOutputs",
    [
        ('sum', float),
        ('product', float),
        ('mlpipeline_ui_metadata', 'UI_metadata'),
        ('mlpipeline_metrics', 'Metrics')
    ]
):
    """ example output function that demonstrates how to reutrn multiple values"""
    
    sum_value = a + b
    product_value = a * b
    
    metadata = {
        'outputs': [{
            'type': 'tensorboard',
            'source': 'gs://ml-pipeline-dataset/tensorboard-train',
        }]
    }
    
    #export two metric
    metrics = {
        'metrics': [
            {
                'name': 'sum',
                'numberValue': float(sum_value),
            },
            {
                'name': 'product',
                'numberValue': float(product_value),
            }
        ]
    }
    from collections import namedtuple
    example_output = namedtuple(
        'ExampleOutputs',
        ['sum', 'product', 'mlpipeline_ui_metadata', 'mlpipeline_metrics']
    )
    
    return example_output(sum_value, product_value, metadata, metrics)

In [12]:
def split_text_lines(
    source_path: kfp.components.InputPath(str),
    odd_lines_path: kfp.components.OutputPath(str),
    even_lines_path: kfp.components.OutputPath(str)
):
    """Splits a text file into two files, with even lines going to one file
    and odd lines to the other"""
    with open(source_path, 'r') as reader:
        with open(odd_lines_path, 'w') as odd_writer:
            with open(even_lines_path, 'w') as even_writer:
                while True:
                    line = reader.readline()
                    if line == "":
                        break
                    odd_writer.write(line)
                    line = reader.readline()
                    if line == "":
                        break
                    even_writer.wirte(line)

In [13]:
def my_divmod(
    dividend: float,
    divisor: float
) -> NamedTuple(
    'MyDivmodOutput',
    [
        ('quotient', float),
        ('remainder', float),
        ('mlpipeline_ui_metadata', 'UI_metadata'),
        ('mlpipeline_metrics', 'Metrics')
    ]
):
    """divides two number and calculate the quotient and reamainder"""
    
    import numpy as np
    
    def divmod_helper(dividend, divisor):
        return np.divmod(dividend, divisor)
    
    (quotient, remainder) = divmod_helper(dividend, divisor)
    
    from tensorflow.python.lib.io import file_io
    import json
    
    metadata = {
        'outputs': [{
            'type': 'tensorboard',
            'source': 'gs://ml-pipeline-dataset/tensorboard-train',
        }]
    }
    
    metrics = {
        'metrics': [{
            'name': 'quotient',
            'numberValue': float(quotient),
        },
        {
          'name': 'remainder',
            'numbervalue': float(remainder),
        }]
        
    }
    from collections import namedtuple
    divmod_output = namedtuple('MyDivmodOutput',
                             ['quotient', 'remainder', 'mlpipeline_ui_metadata', 'mlpipeline_metrics'])
    
    return divmod_output(quotient, remainder, json.dumps(metadata), json.dumps(metrics))

In [14]:
# test function or unit tests
my_divmod(100, 7)

MyDivmodOutput(quotient=14, remainder=2, mlpipeline_ui_metadata='{"outputs": [{"type": "tensorboard", "source": "gs://ml-pipeline-dataset/tensorboard-train"}]}', mlpipeline_metrics='{"metrics": [{"name": "quotient", "numberValue": 14.0}, {"name": "remainder", "numbervalue": 2.0}]}')

In [15]:
divmod_op = create_component_from_func(
    my_divmod,
    base_image='tensorflow/tensorflow:1.11.0-py3'
)

In [16]:
@dsl.pipeline(
    name="Calcilation pipeline",
    description="An example pipeline tha perform arithmetic calciations"
)

def calc_pipeline(
a='1', b='7', c='17',
):
    add_task = add_op(a, 4)
    
    divmod_task = divmod_op(add_task.output, b)
    
    result_task = add_op(divmod_task.outputs['quotient'], c)

In [17]:
# run pipeline
arguments = { 'a': '7', 'b': '8'}

client.create_run_from_pipeline_func(calc_pipeline,
                                     arguments=arguments)

RunPipelineResult(run_id=57c1182d-0b57-45bc-9adb-017f16f2bd93)